In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
sys.path.append("C:\\Users\\user\\Documents\\Workspace\\MeronymNet-PyTorch\\src")

In [4]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader as GraphDataloader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [5]:
from losses import BoxVAE_losses as loss
from components.DenseAutoencoder import DenseAutoencoder
from components.DenseAutoencoder import Decoder

In [6]:
import matplotlib.pyplot as plt

In [7]:
from mask_generation import masked_sketch

In [8]:
colors = [(1, 0, 0),
          (0.737, 0.561, 0.561),
          (0.255, 0.412, 0.882),
          (0.545, 0.271, 0.0745),
          (0.98, 0.502, 0.447),
          (0.98, 0.643, 0.376),
          (0.18, 0.545, 0.341),
          (0.502, 0, 0.502),
          (0.627, 0.322, 0.176),
          (0.753, 0.753, 0.753),
          (0.529, 0.808, 0.922),
          (0.416, 0.353, 0.804),
          (0.439, 0.502, 0.565),
          (0.784, 0.302, 0.565),
          (0.867, 0.627, 0.867),
          (0, 1, 0.498),
          (0.275, 0.51, 0.706),
          (0.824, 0.706, 0.549),
          (0, 0.502, 0.502),
          (0.847, 0.749, 0.847),
          (1, 0.388, 0.278),
          (0.251, 0.878, 0.816),
          (0.933, 0.51, 0.933),
          (0.961, 0.871, 0.702)]
colors = (np.asarray(colors)*255)
canvas_size = 660

def plot_bbx(bbx):
    bbx = bbx*canvas_size
    canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
    for i, coord in enumerate(bbx):
        x_minp, y_minp,x_maxp , y_maxp = coord[:4]
        if [x_minp, y_minp, x_maxp, y_maxp]!=[0,0,0,0]:
            cv2.rectangle(canvas, (int(x_minp), int(y_minp)), (int(x_maxp) , int(y_maxp) ), colors[i], 6)
    return canvas

def plot_obj_bbx(bbx):
    bbx = bbx*canvas_size
    canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
    x_minp, y_minp,x_maxp , y_maxp = bbx
    cv2.rectangle(canvas, (int(x_minp), int(y_minp)), (int(x_maxp) , int(y_maxp) ), colors[0], 6)
    return canvas

# def plot_bbx(bbx):
#     bbx = bbx*canvas_size
#     canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
#     for i, coord in enumerate(bbx):
#         x, y, w ,h = coord
#         if [x, y, w ,h]!=[0,0,0,0]:
#             cv2.rectangle(canvas, (int(x), int(y)), (int(x + w) , int(y + h) ), colors[i], 6)
#     return canvas


In [9]:
def inference_obj(decoder, node_data):

    decoder.cuda()
    z_latent = torch.normal(torch.zeros([batch_size,latent_dims])).cuda()
    nodes = torch.reshape(nodes,(batch_size,decoder.num_nodes))
    obj_class = torch.reshape(obj_class,(batch_size, decoder.class_size))
    conditioned_z = torch.cat([nodes, z_latent],dim=-1)
    conditioned_z = torch.cat([obj_class, conditioned_z],dim=-1)
    
    op = decoder(conditioned_z)
        
    return op[0], op[1], z_latent


def inference(decoder, nodes, obj_class, latent_dims, batch_size):

    decoder.cuda()
    z_latent = torch.normal(torch.zeros([batch_size,latent_dims])).cuda()
    nodes = torch.reshape(nodes,(batch_size,decoder.num_nodes))
    obj_class = torch.reshape(obj_class,(batch_size, decoder.class_size))
    conditioned_z = torch.cat([nodes, z_latent],dim=-1)
    conditioned_z = torch.cat([obj_class, conditioned_z],dim=-1)
    
    op = decoder(conditioned_z)
        
    return op[0], op[1], z_latent

In [10]:
file_postfix = '_obj_boundary'
outfile = 'D:/meronym_data/X_train'+file_postfix+'.np'
with open(outfile, 'rb') as pickle_file:
    X_train = pickle.load(pickle_file)

# outfile = 'D:/meronym_data/class_v'+file_postfix+'.np'
# with open(outfile, 'rb') as pickle_file:
#     class_v = pickle.load(pickle_file)

# outfile = 'D:/meronym_data/adj_train'+file_postfix+'.np'
# with open(outfile, 'rb') as pickle_file:
#     adj_train = pickle.load(pickle_file)

outfile = 'D:/meronym_data/X_train_val'+file_postfix+'.np'
with open(outfile, 'rb') as pickle_file:
    X_train_val = pickle.load(pickle_file)

# outfile = 'D:/meronym_data/class_v_val'+file_postfix+'.np'
# with open(outfile, 'rb') as pickle_file:
#     class_v_val = pickle.load(pickle_file)
    
# outfile = 'D:/meronym_data/adj_train_val'+file_postfix+'.np'
# with open(outfile, 'rb') as pickle_file:
#     adj_train_val = pickle.load(pickle_file)
    
outfile = 'D:/meronym_data/X_test'+file_postfix+'.np'
with open(outfile, 'rb') as pickle_file:
    X_test = pickle.load(pickle_file)

# outfile = 'D:/meronym_data/adj_test'+file_postfix+'.np'
# with open(outfile, 'rb') as pickle_file:
#     adj_test = pickle.load(pickle_file)
    
# outfile = 'D:/meronym_data/class_v'+file_postfix+'.np'
# with open(outfile, 'rb') as pickle_file:
#     class_v_test = pickle.load(pickle_file)

In [11]:
def iou(true, pred):
    zero = torch.tensor([0.0]).cuda()

    x1g, y1g, x2g, y2g = torch.tensor_split(true, 4, dim=-1)
    x1, y1, x2, y2 = torch.tensor_split(torch.squeeze(pred), 4, dim=-1)
    
    xA = torch.maximum(x1g, x1)
    yA = torch.maximum(y1g, y1)
    xB = torch.minimum(x2g, x2)
    yB = torch.minimum(y2g, y2)
    
    w, h = x2g-x1g, y2g-y1g
    
    
    interArea = torch.multiply(torch.maximum(zero,(xB - xA)), 
                                   torch.maximum(zero, (yB - yA)))
    boxAArea = torch.multiply(torch.maximum(zero, (x2g - x1g)),
                              torch.maximum(zero, (y2g - y1g)))
    boxBArea = torch.multiply(torch.maximum(zero, (x2 - x1)),
                                  torch.maximum(zero,(y2 - y1)))
    unionArea = boxAArea + boxBArea - interArea
    
    iouk = interArea / unionArea
    
    return iouk
    

In [12]:
X_train[X_train<=0] = 0
X_train_val[X_train_val<=0] = 0
X_test[X_test<=0] = 0

X_train[X_train>=1] = 1
X_train_val[X_train_val>=1] = 1
X_test[X_test>=1] = 1

In [13]:
print(X_train.shape)
print(X_train_val.shape)
print(X_test.shape)

(50910, 4)
(6720, 4)
(10380, 4)


In [14]:
random.seed(100)
train_idx = np.random.randint(1,len(X_train),len(X_train))
val_idx = np.random.randint(1,len(X_train_val),len(X_train_val))
test_idx = np.random.randint(1,len(X_test),len(X_test))

In [15]:
batch_size = 128
seed = 345

torch.manual_seed(seed)

validation = True
if validation:
    train_list =[]
    
    batch_train_loader = data_utils.DataLoader(torch.from_numpy(np.array(X_train)).cuda().float(), batch_size=batch_size)
    batch_val_loader = data_utils.DataLoader(torch.from_numpy(np.array(X_train_val)).cuda().float(), batch_size=batch_size)
else:
    batch_train_loader = data_utils.DataLoader(torch.from_numpy(np.array(np.concatenate([X_train,X_train_val], axis=0))).cuda().float(), batch_size=batch_size)
    batch_val_loader = data_utils.DataLoader(torch.from_numpy(X_test).cuda().float(), batch_size=batch_size)
    

In [16]:
del train_list
del X_train
del X_train_val

NameError: name 'class_v' is not defined

In [17]:
idx = 0
for data in batch_train_loader:
    idx+=1
    print(data.shape)
    if idx==3:
        break

torch.Size([128, 4])
torch.Size([128, 4])
torch.Size([128, 4])


In [28]:
latent_dims = 64
num_nodes = 1
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 50
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.000018
hidden1 = 32
hidden2 = 16
hidden3 = 128
adaptive_margin = True
fine_tune_box = False
output_log = False
area_encoding = False
run_prefix = "obj_box_vae"


In [29]:
import gc
gc.collect()

103

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [31]:
reconstruction_loss_arr = []
kl_loss_arr = []
bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

vae = DenseAutoencoder(latent_dims,
                     bbx_size,
                     hidden1,
                     hidden2,
                    )
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150], gamma=0.5)
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

for epoch in range(nb_epochs):
    
    if epoch==50:
        fine_tune_box = True
    batch_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        node_data_true = train_data
        for param in vae.parameters():
            param.grad=None
        
        output = vae(node_data_true)
        node_data_pred = output
        bbox_loss = loss.obj_bbox_loss(pred_box=node_data_pred, true_box=node_data_true)
        reconstruction_loss = bbox_loss
            
        reconstruction_loss.backward()
        
        optimizer.step()
        
        i+=1
      
        batch_loss += reconstruction_loss
        
        if i%50==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            
    scheduler.step()
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_obj_bbx(node_data_true[0]).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_obj_bbx((node_data_pred[0]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        node_data_true = train_data
        
        output = vae(node_data_true)
        
        node_data_pred = output
        bbox_loss = loss.obj_bbox_loss(pred_box=node_data_pred, true_box=node_data_true)
    
        reconstruction_loss = bbox_loss*1000
        
        batch_loss += reconstruction_loss
    
    image = plot_obj_bbx(node_data_true[0]).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_obj_bbx(node_data_pred[0].detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(100):    
    image = plot_obj_bbx(node_data_true[i]).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_obj_bbx(node_data_pred[i].detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

50
100
150
200
250
300
350
[1,   399] loss: 0.902
50
100
150
200
250
300
350
[2,   399] loss: 0.341
50
100
150
200
250
300
350
[3,   399] loss: 0.219
50
100
150
200
250
300
350
[4,   399] loss: 0.207
50
100
150
200
250
300
350
[5,   399] loss: 0.189
50
100
150
200
250
300
350
[6,   399] loss: 0.160
50
100
150
200
250
300
350
[7,   399] loss: 0.118
50
100
150
200
250
300
350
[8,   399] loss: 0.086
50
100
150
200
250
300
350
[9,   399] loss: 0.080
50
100
150
200
250
300
350
[10,   399] loss: 0.079
50
100
150
200
250
300
350
[11,   399] loss: 0.078
50
100
150
200
250
300
350
[12,   399] loss: 0.078
50
100
150
200
250
300
350
[13,   399] loss: 0.077
50
100
150
200
250
300
350
[14,   399] loss: 0.076
50
100
150
200
250
300
350
[15,   399] loss: 0.076
50
100
150
200
250
300
350
[16,   399] loss: 0.075
50
100
150
200
250
300
350
[17,   399] loss: 0.074
50
100
150
200
250
300
350
[18,   399] loss: 0.073
50
100
150
200
250
300
350
[19,   399] loss: 0.072
50
100
150
200
250
300
350
[20,   399] l

IndexError: index 94 is out of bounds for dimension 0 with size 94

In [ ]:
reconstruction_loss

In [32]:
#testing loop
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

class_dict = ['cow', 'sheep', 'bird', 'person', 'cat', 'dog', 'horse', 'aeroplane',
              'motorbike', 'bicycle', 'car']
count_dict = {'cow':0, 'sheep':0, 'bird':0, 'person':0, 'cat':0, 'dog':0, 'horse':0,
              'aeroplane':0, 'motorbike':0, 'bicycle':0, 'car':0}

write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = DenseAutoencoder(latent_dims,
                     bbx_size,
                     hidden1,
                     hidden2,
                    )
vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
pred_boxes = []
classes = []
iou_values = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data
    output = vae(node_data_true)
    node_data_pred_test = output
    pred_boxes.append((node_data_pred_test).detach().to("cpu").numpy())
    iou_values.append(iou(node_data_true, node_data_pred_test).detach().to("cpu").numpy())
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_bbx(node_data_true[j].detach().to("cpu").numpy().astype(float))/255
            pred_image = plot_bbx(node_data_pred_test[j].detach().to("cpu").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

writer.flush()
writer.close()
    


In [33]:

boxes_combined = np.concatenate(iou_values)
print(np.mean(boxes_combined))
# classes_combined = np.concatenate([np.argmax(np.reshape(class_val,
#                                                         (int(class_val.shape[0]/7),7)),axis=-1) 
#                                    for class_val in classes])

# for i in range(7):
#     idxes = np.where(classes_combined==i)[0]
#     mean_iou = np.mean(boxes_combined[idxes])
#     print('Class '+str(i)+':',mean_iou)

0.9932367


In [ ]:
IOU_all = iou(node_data_true[:,1:], node_data_pred_test)
torch.mean(IOU_all)

In [ ]:
node_data_pred_test.shape

In [ ]:
int(np.argmax(class_true[10*j:10*(j+1)].detach().to('cpu').numpy()).tolist())

In [ ]:
outfile = 'D:/meronym_data/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pred_boxes = np.concatenate(pred_boxes)
    pickle.dump(pred_boxes, pickle_file)
outfile = 'D:/meronym_data/generate_boxesobj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)

In [ ]:
import gc
gc.collect()